<table>
    <tr><td><h1>Bases de Datos para el Big Data</h1></td><td> <img width="284" height="57" src="https://www.unir.net/wp-content/uploads/2019/11/logo.png" alt="UNIR" /> </td></tr>
</table>

<h1>Tema 4 - MongoDB: Agregación</h2>

<hr>

## Uso de MongoDB desde Python

Taller progresivo (principiante → avanzado) centrado en PyMongo. Cada seccion tiene pasos guiados y ejercicios para practicar.

## Ruta del taller
1. Instalacion y configuracion inicial
2. Conexion basica con PyMongo
3. Creacion e insercion de documentos
4. Lectura y consultas simples
5. Actualizacion y eliminacion
6. Indices y rendimiento
7. Consultas avanzadas y agregaciones
8. Manejo de errores y validacion
9. Variables de entorno para credenciales
10. Pruebas basicas CRUD
11. Operaciones en lotes y transacciones
12. Ejercicios avanzados

## 1) Instalacion y configuracion inicial
- Instala MongoDB local o crea un cluster en Atlas (usuario de lectura/escritura).
- Instala dependencias Python con pip.
- Verifica version de PyMongo para asegurar compatibilidad.

In [ ]:
# Ejecuta solo si faltan dependencias
!pip install pymongo python-dotenv

## 2) Conexion basica a MongoDB con PyMongo
- Usa URI local (`mongodb://localhost:27017`) o de Atlas con usuario/contraseña.
- Configura un pool moderado para demos: `maxPoolSize`, `minPoolSize`, timeouts cortos.
- Verifica la conexion con `ping` y lista bases/colecciones para confirmar acceso.

In [ ]:
import os, pathlib
from pprint import pprint
from pymongo import MongoClient
from pymongo.errors import ConnectionFailure

BASE_DIR = pathlib.Path().resolve()
DATA_DIR = BASE_DIR / "data"

def get_client(uri=None, **kwargs):
    uri = uri or os.getenv("MONGODB_URI", "mongodb://localhost:27017")
    params = {
        "maxPoolSize": 20,
        "minPoolSize": 2,
        "serverSelectionTimeoutMS": 5000,
        "socketTimeoutMS": 5000,
        "connectTimeoutMS": 5000,
    }
    params.update(kwargs)
    client = MongoClient(uri, **params)
    try:
        client.admin.command("ping")
    except ConnectionFailure as exc:
        raise RuntimeError(f"No se pudo conectar a MongoDB en {uri}") from exc
    return client

client = get_client()
print("Ping OK")
print("Bases disponibles:", client.list_database_names())

## 3) Creacion e insercion de documentos
- Usa una BD de demo `LabTestDB` y la coleccion `compras`.
- Ejemplos de `insert_one` y `insert_many` con datos simples para principiantes.

In [ ]:
db = client["LabTestDB"]
db.compras.drop()  # Limpiamos la colección antes de insertar datos

one_result = db.compras.insert_one({
    "cliente_id": "AB001",
    "total": 120.5,
    "estatus": "pendiente",
    "articulos": [{"producto": "teclado", "cantidad": 1, "precio": 120.5}],
})
print("insert_one ->", one_result.inserted_id)

many_result = db.compras.insert_many([
    {"cliente_id": "AB002", "total": 45, "estatus": "entregado"},
    {"cliente_id": "AB003", "total": 80, "estatus": "entregado"},
])
print("insert_many ->", many_result.inserted_ids)

print("Total documentos en compras:", db.compras.count_documents({}))

## 4) Lectura y consultas simples
- `find_one` para un documento.
- `find` con filtro, proyeccion, `limit` y `sort`.

In [ ]:
pprint(db.compras.find_one({}, {"cliente_id": "AB001", "total": 1, "_id": 0}))

cursor = db.compras.find(
    {"estatus": "entregado"},
    {"cliente_id": 1, "total": 1, "_id": 0}
).sort("total", -1).limit(5)
print("Compras entregadas (top 5 por total):")
for doc in cursor:
    pprint(doc)

## 5) Actualizacion y eliminacion
- `update_one` y `update_many` con `$set` y `$inc`.
- `delete_one` y `delete_many` para limpiar datos de prueba.

In [ ]:
db.compras.update_one({"cliente_id": "AB001"}, {"$set": {"estatus": "en_proceso"}, "$inc": {"total": 5}})

result = db.compras.update_many({"estatus": "entregado"}, {"$inc": {"total": 1}})
print("update_many matched/modified:", result.matched_count, result.modified_count)

print("Antes de borrar:", db.compras.count_documents({}))
db.compras.delete_one({"cliente_id": "AB002"})
db.compras.delete_many({"estatus": "pendiente"})
print("Despues de borrar:", db.compras.count_documents({}))

## 6) Indices y rendimiento
- Crea indice simple por `cliente_id` y compuesto por `(estatus, total)`.
- Revisa indices con `list_indexes`.
- Observa plan de ejecucion con `explain()` para notar el uso del indice.

In [ ]:
db.compras.create_index("cliente_id")
db.compras.create_index([("estatus", 1), ("total", -1)])

print("Indices definidos:")
for idx in db.compras.list_indexes():
    pprint(idx)

plan = db.compras.find({"estatus": "entregado"}).sort("total", -1).explain()["queryPlanner"]
print("Plan (resumen):", plan.get("winningPlan", {}))

## 7) Consultas avanzadas y agregaciones
- Construye pipelines con `$match`, `$project`, `$group`, `$sort`.
- Agrega operadores intermedios: `$unwind`, `$lookup`, `$bucket`, `$facet`.
- Usaremos datos de demo; puedes sustituir por los JSON reales en `data/` con el helper siguiente.

In [ ]:
from bson import json_util

def load_json_collection(json_file: str, collection_name: str, drop_first: bool = True, limit = None):
    path = DATA_DIR / json_file
    with open(path, "r", encoding="utf-8") as f:
        docs = json_util.loads(f.read())
    if limit:
        docs = docs[:limit]
    if drop_first:
        db[collection_name].drop()
    if docs:
        db[collection_name].insert_many(docs)
    print(f"{collection_name}: {len(docs)} docs cargados de {json_file}")


In [ ]:
# Descomenta para cargar datasets completos de data/
load_json_collection("LabTestDB.clientes.json", "clientes")
# load_json_collection("LabTestDB.compras.json", "compras")

In [ ]:
def run_pipeline(coll, pipeline, limit=5):
    return list(coll.aggregate(pipeline))[:limit]

pipelines = {
    "ventas_por_estatus": [
        {"$group": {"_id": "$estatus", "monto": {"$sum": "$total"}, "compras": {"$sum": 1}}},
        {"$sort": {"monto": -1}},
    ],
    "top_productos": [
        {"$unwind": "$articulos"},
        {"$group": {"_id": "$articulos.producto", "ventas": {"$sum": "$articulos.cantidad"}}},
        {"$sort": {"ventas": -1}},
        {"$limit": 5},
    ],
    "ventas_por_cliente": [
        {"$group": {"_id": "$cliente_id", "compras": {"$sum": 1}, "monto": {"$sum": "$total"}}},
        {"$sort": {"monto": -1}},
    ],
    "bucket_total": [
        {"$bucket": {"groupBy": "$total", "boundaries": [0, 50, 100, 200, 500], "default": "500+", "output": {"count": {"$sum": 1}}}}
    ],
}

for name, pipeline in pipelines.items():
    print(f"--- {name} ---")
    for doc in run_pipeline(db.compras, pipeline):
        pprint(doc)
    print()

## 8) Manejo de errores y validacion
- Captura `ConnectionFailure`, `OperationFailure`.
- Valida datos antes de insertar (campos obligatorios, tipos basicos).
- Usa try/except para mostrar mensajes claros en clase.

In [ ]:
from pymongo.errors import OperationFailure

def validar_compra(doc: dict):
    requeridos = ["cliente_id", "total"]
    for campo in requeridos:
        if campo not in doc:
            raise ValueError(f"Falta campo obligatorio: {campo}")
    if not isinstance(doc.get("total"), (int, float)):
        raise ValueError("total debe ser numero")

try:
    compra = {"cliente_id": "CL_X", "total": "no-num"}
    validar_compra(compra)
    db.compras.insert_one(compra)
except ValueError as e:
    print("Validacion evitada:", e)

try:
    db.command({"collMod": "compras", "validator": {"total": {"$type": "number"}}})
except OperationFailure as e:
    print("Operacion no aplicada (posible falta de permisos en cluster compartido):", e.details.get("errmsg", str(e)))

## 9) Variables de entorno para credenciales
- Nunca hardcodees credenciales.
- Usa `.env` con `python-dotenv` o configura variables del sistema.
- Lee `MONGODB_URI` y crea el cliente con ella.

In [ ]:
from dotenv import load_dotenv
load_dotenv()
uri_env = os.getenv("MONGODB_URI", "mongodb://localhost:27017")
print("URI cargada:", uri_env)
client_env = get_client(uri_env)
print("Ping con URI de entorno OK")

## 10) Pruebas basicas de operaciones CRUD
- Usa `unittest` para validar insercion/lectura/actualizacion/borrado en una coleccion temporal.
- Buen habito para verificar talleres antes de la clase.

In [ ]:
import unittest

class TestCRUD(unittest.TestCase):
    def setUp(self):
        self.coll = db["test_crud"]
        self.coll.drop()

    def test_insert_and_find(self):
        self.coll.insert_one({"x": 1, "y": "a"})
        doc = self.coll.find_one({"x": 1})
        self.assertEqual(doc["y"], "a")

    def test_update(self):
        self.coll.insert_one({"x": 1, "total": 10})
        self.coll.update_one({"x": 1}, {"$inc": {"total": 5}})
        doc = self.coll.find_one({"x": 1})
        self.assertEqual(doc["total"], 15)

    def test_delete(self):
        self.coll.insert_many([{"x": i} for i in range(3)])
        self.coll.delete_many({"x": {"$gte": 1}})
        self.assertEqual(self.coll.count_documents({}), 1)

suite = unittest.TestLoader().loadTestsFromTestCase(TestCRUD)
unittest.TextTestRunner().run(suite)

## 11) Operaciones en lotes y transacciones simples
- `bulk_write` mezcla inserciones, updates y deletes en una sola llamada.
- Demostracion de transaccion (requiere replica set/Atlas). Si no hay soporte, manejar excepcion.

In [ ]:
from pymongo import InsertOne, UpdateOne, DeleteOne
from pymongo.errors import ConfigurationError

ops = [
    InsertOne({"cliente_id": "bulk", "total": 10}),
    UpdateOne({"cliente_id": "bulk"}, {"$set": {"estatus": "ok"}}),
    DeleteOne({"cliente_id": "CL_demo"}),
]
result = db.compras.bulk_write(ops)
print("bulk_write -> inserted", result.inserted_count, "updated", result.modified_count, "deleted", result.deleted_count)

try:
    with client.start_session() as s:
        with s.start_transaction():
            db.compras.insert_one({"cliente_id": "tx", "total": 5}, session=s)
            db.compras.update_one({"cliente_id": "tx"}, {"$inc": {"total": 5}}, session=s)
        print("Transaccion aplicada")
except (ConfigurationError, OperationFailure) as e:
    print("Transaccion no soportada en este cluster:", str(e))

<div class="alert alert-block alert-info">
<i class="fa fa-info-circle"></i>

### Nota sobre transacciones
- **Por que puede fallar aqui:** Las transacciones multi-documento requieren que el servidor sea un **replica set** (local iniciado con `--replSet` + `rs.initiate()` o un cluster de Atlas). En un servidor standalone o un sandbox sin replica set, PyMongo lanza `ConfigurationError` o `OperationFailure`, que es el mensaje que ves (“Transaccion no soportada en este cluster”).
- **Que hace el ejemplo:** Intenta abrir una sesion y una transaccion, ejecutar dos operaciones (insert + update) y confirmar. Si el entorno no soporta transacciones, capturamos la excepcion para que la clase no se detenga.
- **Como habilitarlo localmente:** inicia `mongod --replSet rs0`, luego en el shell: `rs.initiate()`; reconecta PyMongo apuntando a ese mongod. En Atlas, basta con usar un cluster (no el free tier mas antiguo sin transacciones) y la URI SRV.
- **Documentacion oficial:**
  - Transacciones en MongoDB: https://www.mongodb.com/docs/manual/core/transactions/
  - Transacciones con PyMongo: https://www.mongodb.com/docs/drivers/pymongo/current/faq/#how-do-i-use-transactions

</div>




## GeoJSON: indice 2dsphere y busqueda por proximidad
- Cargar `listings.csv` de `data/` y crear campo `location` tipo `Point`.
- Crear indice `2dsphere` y consultar con `$near` / `$geoNear`.

En este taller, utilizarás un conjunto de datos público para trabajar con MongoDB.

El conjunto de datos pertenece a una colección pública propiedad de __Airbnb__, la cual hace parte de su apuesta _Open Data_.
En el siguiente enlace podrás obtener más información de la misma:

> * <i class="fa fa-link" style="color:#113D68"></i> [Inside Airbnb](http://insideairbnb.com)

__Contenido de los datos__: información de los anuncios disponibles en la web de Airbnb publicados en algunas ciudades donde opera dicha compañía.

In [ ]:
import pandas as pd

def load_listings(csv_file: str = "listings.csv"):
    path = DATA_DIR / csv_file
    df = pd.read_csv(path)
    df = df.dropna(subset=["latitude", "longitude"])
    df["price"] = pd.to_numeric(df["price"], errors="coerce")
    df = df.dropna(subset=["price"])
    df["location"] = df.apply(
        lambda r: {"type": "Point", "coordinates": [float(r["longitude"]), float(r["latitude"])]}, axis=1
    )
    docs = (
        df[["id", "name", "neighbourhood", "neighbourhood_group", "room_type", "price", "location"]]
        .rename(columns={"neighbourhood": "barrio", "neighbourhood_group": "distrito"})
        .to_dict("records")
    )
    db.alojamientos.drop()
    if docs:
        db.alojamientos.insert_many(docs)
    db.alojamientos.create_index([("location", "2dsphere")])
    print("Alojamientos cargados:", len(docs))

load_listings()

madrid_centro = {"type": "Point", "coordinates": [-3.7038, 40.4168]}
print("Cerca con $near:")
for doc in db.alojamientos.find(
    {"location": {"$near": {"$geometry": madrid_centro, "$maxDistance": 2000}}},
    {"_id": 0, "name": 1, "price": 1, "distrito": 1},
).limit(5):
    pprint(doc)

print("\nCon $geoNear:")
for doc in db.alojamientos.aggregate([
    {"$geoNear": {"near": madrid_centro, "distanceField": "dist_m", "maxDistance": 2000, "spherical": True}},
    {"$project": {"_id": 0, "name": 1, "price": 1, "distrito": 1, "dist_m": 1}},
    {"$sort": {"dist_m": 1}},
    {"$limit": 5},
]):
    pprint(doc)

### Visualizar resultados geo en un mapa con Folium
- Objetivo: pintar en un mapa los alojamientos recuperados con una consulta geoespacial.
- Usamos `folium` para renderizar puntos cercanos al centro, limitando distancia y precio.
- Documentacion folium: https://python-visualization.github.io/folium/latest/

In [ ]:
# Ejecuta solo si no tienes folium instalado
# !pip install folium

In [ ]:
import folium
from IPython.display import HTML

madrid_centro = (40.4168, -3.7038)
max_distance_m = 1500
max_price = 120

query = {
    "location": {
        "$near": {
            "$geometry": {"type": "Point", "coordinates": [madrid_centro[1], madrid_centro[0]]},
            "$maxDistance": max_distance_m,
        }
    },
    "price": {"$lt": max_price},
}

cursor = db.alojamientos.find(query, {"_id": 0, "name": 1, "price": 1, "location": 1, "distrito": 1}).limit(50)
results = list(cursor)
print(f"Puntos encontrados: {len(results)}")

mapa = folium.Map(location=madrid_centro, zoom_start=13, tiles="CartoDB positron")
for doc in results:
    coords = doc["location"]["coordinates"]
    folium.CircleMarker(
        location=[coords[1], coords[0]],
        radius=5,
        popup=f"{doc.get('name','sin nombre')} | €{doc.get('price','?')}",
        color="#0b7285",
        fill=True,
        fill_opacity=0.8,
    ).add_to(mapa)

if results:
    lats = [doc["location"]["coordinates"][1] for doc in results]
    lngs = [doc["location"]["coordinates"][0] for doc in results]
    mapa.fit_bounds([[min(lats), min(lngs)], [max(lats), max(lngs)]])
else:
    print("Sin resultados dentro de los filtros; se muestra el centro por defecto.")

HTML(mapa._repr_html_())

### Alternativa rapida: mapa con Plotly (OpenStreetMap, sin token)
- Usa los mismos datos de `alojamientos` y tiles libres `open-street-map` (no requiere credenciales).
- Ajusta los filtros `max_distance_m` y `max_price` si necesitas mas puntos.
- Si no ves nada, asegúrate de ejecutar antes la celda `load_listings()`.


In [ ]:
#!pip install plotly

In [ ]:
import pandas as pd
import plotly.express as px

# Usa los mismos filtros que en el ejemplo de folium
madrid_centro = (40.4168, -3.7038)
max_distance_m = 1500
max_price = 120

query = {
    "location": {
        "$near": {
            "$geometry": {"type": "Point", "coordinates": [madrid_centro[1], madrid_centro[0]]},
            "$maxDistance": max_distance_m,
        }
    },
    "price": {"$lt": max_price},
}

results = list(db.alojamientos.find(query, {"_id": 0, "name": 1, "price": 1, "location": 1, "distrito": 1}).limit(200))
print(f"Puntos encontrados: {len(results)}")

if not results:
    print("Sin resultados con estos filtros; ajusta max_distance_m o max_price.")
else:
    df = pd.DataFrame(results)
    df["lat"] = df["location"].apply(lambda x: x["coordinates"][1])
    df["lon"] = df["location"].apply(lambda x: x["coordinates"][0])

    fig = px.scatter_map(
        df,
        lat="lat",
        lon="lon",
        hover_name="name",
        hover_data={"price": True, "distrito": True, "lat": False, "lon": False},
        color="price",
        color_continuous_scale="Blues",
        size_max=12,
        zoom=12,
        height=520,
    )
    fig.update_traces(marker=dict(size=9, opacity=0.9))
    fig.update_layout(mapbox_style="open-street-map", margin={"l":0,"r":0,"t":0,"b":0})
    fig.show()


## 12) Repasa con algunos ejercicios
**Basico:**
- Crear coleccion `clientes_demo`, insertar 3 clientes y consultarlos con proyeccion.
- Agregar un indice por `email` y verificarlo.

**Intermedio:**
- Cargar `LabTestDB.compras.json` (usar `limit` si es pesado) y calcular top 5 productos por ingreso (`$unwind`, `$group`, `$sort`).
- Unir `compras` y `clientes` con `$lookup` para obtener compras por ciudad.
- Crear buckets de tickets: [0-100, 100-300, 300-1000, 1000+] con `$bucket`.

**Avanzado:**
- Definir un esquema de validacion que exija `total` numerico y `cliente_id` string; probar insercion valida e invalida.
- Disenar un indice compuesto optimo para consultas frecuentes (`cliente_id`, `estatus`, `total`).
- Construir un pipeline `$facet` con dos salidas: ventas por metodo de pago y ventas por mes (`$dateToString`).
- Geo: modificar la consulta `$geoNear` para limitar a alojamientos < 100 euros y mostrar solo `name`, `price`, `dist_m`.

### Otros ejercicios propuestos

1) **Insercion y validacion rapida**: Inserta 3 clientes con campos minimos (`_id`, `nombre`, `email`). Ejemplo: `db.clientes_demo.insert_one({"_id":"C1","nombre":"Ana","email":"ana@example.com"})`.

2) **Filtro y proyeccion**: Consulta clientes que vivan en `Ciudad de México` proyectando solo `nombre` y `email`. Ejemplo: `db.clientes.find({"direccion.ciudad":"Ciudad de México"},{"_id":0,"nombre":1,"email":1})`.

3) **Update con $inc**: Incrementa `puntos_fidelidad` en +50 para clientes `estatus:"activo"`. Ejemplo: `db.clientes.update_many({"estatus":"activo"},{"$inc":{"puntos_fidelidad":50}})`.

4) **Borrado selectivo**: Elimina compras con `estatus:"pendiente"` y `total<20`. Ejemplo: `db.compras.delete_many({"estatus":"pendiente","total":{"$lt":20}})`.

5) **Agregacion basica**: Total de ventas por estatus. Pipeline sugerido: `[{$group:{_id:"$estatus", total:{$sum:"$total"}, compras:{$sum:1}}},{$sort:{total:-1}}]`.

6) **Top productos**: Desenrolla articulos y ordena por unidades vendidas. Pipeline: `[{$unwind:"$articulos"},{$group:{_id:"$articulos.producto", unidades:{$sum:"$articulos.cantidad"}}},{$sort:{unidades:-1}},{$limit:5}]`.

7) **Lookup clientes-compras**: Une compras con clientes y agrupa por ciudad. Pipeline: `[{$lookup:{from:"clientes",localField:"cliente_id",foreignField:"_id",as:"cli"}},{$unwind:"$cli"},{$group:{_id:"$cli.direccion.ciudad", compras:{$sum:1}, monto:{$sum:"$total"}}},{$sort:{monto:-1}}]`.

8) **Bucket de tickets**: Segmenta compras por `total` en rangos [0-100, 100-300, 300-1000, 1000+]. Pipeline: `[{$bucket:{groupBy:"$total", boundaries:[0,100,300,1000], default:"1000+", output:{compras:{"$sum":1}, promedio:{"$avg":"$total"}}}}]`.

9) **Facet doble**: Una sola pasada que devuelva (a) ventas por metodo y (b) ventas por mes. Pipeline: `[{$facet:{por_metodo:[{$group:{_id:"$metodo_pago", monto:{$sum:"$total"}}},{$sort:{monto:-1}}], por_mes:[{$addFields:{ym:{$dateToString:{format:"%Y-%m", date:"$fecha"}}}},{$group:{_id:"$ym", monto:{$sum:"$total"}, compras:{$sum:1}}},{$sort:{_id:1}}]}}]`.

10) **Geo + filtro**: Alojamientos a 1 km del centro y precio < 100. Pipeline: `[{$geoNear:{near:{type:"Point",coordinates:[-3.7038,40.4168]}, distanceField:"dist_m", maxDistance:1000, spherical:true}},{$match:{price:{"$lt":100}}},{$project:{_id:0,name:1,price:1,dist_m:1}},{$sort:{dist_m:1}},{$limit:5}]`.

Usa estos ejemplos como plantillas: copia, pega y ajusta campos/valores según tu demo. Cada ejercicio se puede ejecutar tal cual (si existen los campos) o adaptarlo a los datasets cargados en `data/`. Para práctica adicional, pide al alumnado explicar qué hace cada etapa antes de ejecutarla.